# Geographic data visualization with cartograms

A cartogram is a transformation of a map such that the geographic size, or area, of different regions (e.g. countries) is proportional to an underlying quantity (e.g. population) we wish to present.
![earliest_cartogram](https://drive.google.com/uc?id=1FVqs6VCyzJ-faXTa8E8bT5YNrEHvTvVz)

![world_map](https://drive.google.com/uc?id=1OaUswBPNVbF6aWB9NsbYJVPkIN-NmXgm)

## Gastner-Newman diffusion-based cartograms
> Gastner, M. T., & Newman, M. E. (2004). Diffusion-based method for producing density-equalizing maps. *Proceedings of the National Academy of Sciences*, 101(20), 7499-7504. [doi:10.1038/s41467-017-01190-3](doi:10.1038/s41467-017-01190-3).

If the areas of different regions are proportional to population, the population density on the cartogram will be uniform everywhere. Thus, the evolution of a geographic map to a cartogram can be modeled as a diffusive process - the target quantity (population) diffuses in space away from regions of high concentration and toward regions of low concentration, until the amount per unit area is uniform and a steady-state is reached.

In the Gastner-Newman model, the quantity of interest is represented by $\rho$, and obeys the diffusion equation:

$$
\frac{\partial\rho}{\partial{t}} = \nabla^2\rho,
$$

where $\nabla^2$ is the Laplacian

$$
\nabla^2 = \frac{\partial^2}{\partial{x^2}} + \frac{\partial^2}{\partial{y^2}}.
$$

The velocity field induced by the gradient in $\rho$ is given by

$$
\mathbf{v}(\mathbf{r},t) = -\frac{\nabla\rho}{\rho}
$$

If our map is discretized into grid points, each of these points will then passively drift in response to the velocity field. One way to track this drift is to introduce a reference map $\mathbf{X}(\mathbf{x},t)$. For a grid point at position $\mathbf{x}$ at time $t_f$, $\mathbf{X}(\mathbf{x},t_f)$ gives the position of this point at time $t=0$, as shown in the following figure:

![ref_map](https://drive.google.com/uc?id=1eX8s-zuIMNoaa72mGyy9cHD6ZQq8roYE)

The reference map evolves according to the advection equation,

$$
\frac{\partial\mathbf{X}}{\partial{t}} = -(\mathbf{v}\cdot\nabla)\mathbf{X}
$$

Thus we have a system of PDEs to solve simultaneously:

$$
\begin{align}
\frac{\partial\rho}{\partial{t}} &= \nabla^2\rho, \\
\frac{\partial\mathbf{X}}{\partial{t}} &= -(\mathbf{v}\cdot\nabla)\mathbf{X},
\end{align}
$$

where $\mathbf{v}$ is computed from

$$
\mathbf{v}(\mathbf{r},t) = -\frac{\nabla\rho}{\rho}
$$

at each step.
In this demo, we integrate this PDE using the backward Euler method. The Laplacian is discretized according to the centered finite difference scheme:

$$
\nabla^2\rho_{ij} = \frac{\rho_{i+1,j}+\rho_{i-1,j}+\rho_{i,j+1}+\rho_{i,j-1} - 4\rho_{ij}}{h^2},
$$

and the gradient $\nabla\rho$ in the velocity calculation is likewise discretized according to a centered finite difference scheme:

$$
\nabla\rho_{ij} = \Bigg[\frac{\rho_{i+1,j}-\rho_{i-1,j}}{2h},\frac{\rho_{i,j+1}-\rho_{i,j-1}}{2h}\Bigg].
$$

Since the reference map $\mathbf{X}$ evolves in response to the velocity field, the gradient $\nabla\mathbf{X}_{ij}$ is discretized based on *upwinding* conditions: Given the current velocity $\mathbf{v}_{ij}$, we use a forward finite difference if $\mathbf{v}_{ij}>0$ (treated independently for each component of $\mathbf{v}_{ij})$, and the backward finite difference if $\mathbf{v}_{ij}<0$. This ensures that we follow the direction of information propagation, and thus gives better numerical stability.

Our initial geographic map is visualized as a color image with 3 channels, $R_i$, $G_i$, $B_i$. We can construct the channels of the cartogram using our reference map as

$$
\begin{align}
R(\mathbf{x}) &= R_i(\mathbf{X}(\mathbf{x},t_f)) \\
G(\mathbf{x}) &= G_i(\mathbf{X}(\mathbf{x},t_f)) \\
B(\mathbf{x}) &= B_i(\mathbf{X}(\mathbf{x},t_f)) \\
\end{align}
$$

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from geo_data_vis import load_data, populate_map, make_colormap, plot_map

if not os.path.exists('images'):
        os.makedirs('images')
if not os.path.exists('out'):
        os.makedirs('out')

The ```maps``` directory of this repository contains three binary files: ```small.bin```, ```med.bin```, and ```large.bin```. Each of these stores an array corresponding to a map of the US, at different resolutions. All elements of the array corresponding to a particular state have a unique label that can be used to associate the element with an attribute. Background entries are labeled as 0, and states are labeled alphabetically: Alabama = 1, Alaska = 2, etc. Let's look at one of these files:

In [ ]:
# pick an array of labels to view, and determine its dimensions.
size = 'large'
if size == 'large':
    m=816; n=1216
elif size == 'med':
    m=408; n=608
else:
    m=204; n=304

# load in the labels.
labels = np.fromfile("maps/"+size+".bin", dtype=np.uint8).reshape((m,n))

# plot as an image.
fig = plt.figure(figsize=(10,6))
plt.imshow(labels, cmap=plt.cm.tab10, interpolation='nearest')
plt.axis('off')
plt.show()

The ```data``` directory of this repository contains text files of different attributes by state. Each text file starts with a header line that describes the data, and is followed by one line per state which lists the state name followed by the value of the attribute. By associating the state name with its label on the above map, we can make a map displaying the attribute.

Let's look at an example using elevation data. First we'll select our feature to work with, which should be the name of the corresponding text file. The function ```load_data``` will load the text file, sort the values by state name, and save them to a binary file we will later read in our C++ code.

In [ ]:
# Select a dataset to work with
feature = 'elevation'
header = load_data(feature) # returns the header of our dataset, which can be used as a title.

Next let's choose a map resolution to work with. It's recommended to use the small map for testing, as the simulation time scales pretty rapidly with size. For a rough estimate, ```small``` runs in less than 10 seconds, ```med``` runs in ~1.5 minutes, and ```large``` runs in ~25 minutes!

In [ ]:
# Select a data size ('small', 'med', or 'large'),
# and choose colormap and edge color settings.
size = 'small'

 # Populate a US map given the map size and feature.
feature_map = populate_map(size, feature)

Next we can plot our starting feature map as an image. There are a couple of parameters we can tune:
- the colormap, which can be selected from the matplotlib library, and the range of this colormap that we want to include. Full range is [0,1], and we can truncate from below or from above.
- whether to include white borders around each state, or black borders (one may work better with the chosen colormap).

In [ ]:
colormap = make_colormap(plt.cm.cubehelix, 0.2, 1.)
white_edges = False
fig = plot_map(feature_map, header, colormap, white_edges) # plot the map

Once we are happy with the image, we can save to a file.

In [ ]:
fig.savefig("images/"+feature+"_map_"+size+".png")

Next we compile our C++ code which runs the diffusion simulation to generate our reference map. We won't be editing it, so this only needs to happen once, but recompiling doesn't hurt.

In [ ]:
!g++ -O3 -Wall -ansi -pedantic -fopenmp -o cartogram cartogram.cc

Next, we run the diffusion simulation!

In [ ]:
out = !./cartogram $size $feature
print(out[0])
print(out[1])

The simulation above generates a binary file in the ```out``` directory which stores the final $\rho$ and $\mathbf{X}$ fields. We can then use $\mathbf{X}$ to apply our transformation. The binary file is named according to the feature name, so we can supply the feature name as an extra input to our ```plot_map``` function to apply a transformation this time.

In [ ]:
# Plot the same map with deformation mapping applied:
fig = plot_map(feature_map, header, colormap, white_edges, feature_transform=feature)

Finally, we can save the image to a file.

In [ ]:
fig.savefig("images/"+feature+"_cartogram_"+size+".png")

### Your turn!
The ```data``` directory contains a number of different datasets you can test. However, you should feel free to add your own data file as well! You can use other text files as examples for how to structure your text file. All that's necessary is to include a header line to title your dataset, and list out the 50 states along with their attribute (note D.C. is not included in our map). The entries may be in unsorted order.

> **Note:** All entries must be positive; zero and negative values will cause numerical issues.